In [2]:
import data_sources.scrapfly_scrapers.linkedin as linkedin

# enable scrapfly cache
linkedin.BASE_CONFIG["cache"] = True
linkedin.BASE_CONFIG["debug"] = True


In [3]:
linkedin.BASE_CONFIG

{'asp': True,
 'country': 'US',
 'headers': {'Accept-Language': 'en-US,en;q=0.5'},
 'cache': True,
 'debug': True}

In [4]:
import os
import json
import jmespath
from typing import Dict, List
from urllib.parse import urlencode, quote_plus
from parsel import Selector
from loguru import logger as log
from scrapfly import ScrapeConfig, ScrapflyClient, ScrapeApiResponse

SCRAPFLY = ScrapflyClient(key=os.environ.get("SCRAPFLY_KEY"))


company_result = await SCRAPFLY.async_scrape(ScrapeConfig("https://linkedin.com/company/microsoft", **linkedin.BASE_CONFIG))


with open("./company.json", "w", encoding="utf-8") as file:
    json.dump(company_result, file, indent=2, ensure_ascii=False)    


CRITICAL:root:<-- 401 | Invalid API key.


ApiHttpClientError: <-- 401 | Invalid API key.

In [2]:

print("running Linkedin scrape and saving results to ./results directory")

# profile_data = await linkedin.scrape_profile(
#     urls=[
#         "https://www.linkedin.com/in/williamhgates"
#     ]
# )

from pprint import pprint
# pprint(profile_data)


# with open(output.joinpath("profile.json"), "w", encoding="utf-8") as file:
#     json.dump(profile_data, file, indent=2, ensure_ascii=False)    

# company_data = await linkedin.scrape_company(
#     urls=[
#         "https://linkedin.com/company/microsoft",
#         "https://linkedin.com/company/google",
#         "https://linkedin.com/company/apple"
#     ]
# )
# with open(output.joinpath("company.json"), "w", encoding="utf-8") as file:
#     json.dump(company_data, file, indent=2, ensure_ascii=False)    

# job_search_data = await linkedin.scrape_job_search(
#     # it include other search parameters, refer to the search pages on browser for more details
#     keyword="Python Developer",
#     location="United States",
#     max_pages=3
# )
# with open(output.joinpath("job_search.json"), "w", encoding="utf-8") as file:
#     json.dump(job_search_data, file, indent=2, ensure_ascii=False)    

# job_data = await linkedin.scrape_jobs(
#     urls=[
#         "https://www.linkedin.com/jobs/view/data-center-engineering-operations-engineer-hyd-infinity-dceo-at-amazon-web-services-aws-4017265505",
#         "https://www.linkedin.com/jobs/view/content-strategist-google-cloud-content-strategy-and-experience-at-google-4015776107",
#         "https://www.linkedin.com/jobs/view/sr-content-marketing-manager-brand-protection-brand-protection-at-amazon-4007942181"
#     ]
# )
# with open(output.joinpath("jobs.json"), "w", encoding="utf-8") as file:
#     json.dump(job_data, file, indent=2, ensure_ascii=False)

running Linkedin scrape and saving results to ./results directory


In [3]:
# This fails but I have no idea why... it seems to work when I run it manually
job_results = await linkedin.scrape_job_search(
    keyword="software engineer",
    location="United States",
    max_pages=1
)

pprint(job_results)

2024-09-30 14:05:13.469 | WARNING  | scrapfly_scrapers.linkedin:parse_job_search:168 - Job element is missing some data, skipping
2024-09-30 14:05:13.477 | WARNING  | scrapfly_scrapers.linkedin:parse_job_search:168 - Job element is missing some data, skipping
2024-09-30 14:05:13.480 | INFO     | scrapfly_scrapers.linkedin:scrape_job_search:192 - scraped the first job page, 0 more pages


CancelledError: 

In [13]:
def build_job_search_url(keyword, location, **additional_params):
    params = {
        "keywords": linkedin.quote_plus(keyword),
        "location": location,
        **additional_params
    }

    path = "jobs/search"
    if "start" in params:
        path = "jobs-guest/jobs/api/seeMoreJobPostings/search"

    return f"https://www.linkedin.com/{path}?{linkedin.urlencode(params)}"

url = build_job_search_url("software engineer", "United States")
url

'https://www.linkedin.com/jobs/search?keywords=software%2Bengineer&location=United+States'

In [14]:
from scrapfly import ScrapeConfig, ScrapflyClient
import os

scrape_config = ScrapeConfig(build_job_search_url("software engineer", "United States"), **linkedin.BASE_CONFIG)
scrape_config

client = ScrapflyClient(key=os.environ["SCRAPFLY_KEY"])
response = client.scrape(scrape_config)
response

selector = response.selector
total_results = selector.xpath("//span[contains(@class, 'job-count')]/text()").get()
total_results = int(total_results.replace(",", "").replace("+", "")) if total_results else None

print(f"Total results: {total_results}")

data = []
desired_results = 65

while len(data) < total_results and len(data) < desired_results:
    for element in selector.xpath("//section[contains(@class, 'results-list')]/ul/li"):
        try:
            data.append({
                "title": element.xpath(".//div/a/span/text()").get().strip(),
                "company": element.xpath(".//div/div[contains(@class, 'info')]/h4/a/text()").get().strip(),
                "address": element.xpath(".//div/div[contains(@class, 'info')]/div/span/text()").get().strip(),
                "timeAdded": element.xpath(".//div/div[contains(@class, 'info')]/div/time/@datetime").get(),
                "jobUrl": element.xpath(".//div/a/@href").get().split("?")[0],
                "companyUrl": element.xpath(".//div/div[contains(@class, 'info')]/h4/a/@href").get().split("?")[0],
                "salary": linkedin.strip_text(element.xpath(".//span[contains(@class, 'salary')]/text()").get())
            })
        except AttributeError:
            print("Error parsing in input")
            continue
    
    # fetch more results
    print("Fetching more results")
    response = client.scrape(ScrapeConfig(build_job_search_url("software engineer", "United States", start=len(data)), **linkedin.BASE_CONFIG))

print(f"Fetched {len(data)} results")

Total results: 169000
Error parsing in input
Fetching more results
Error parsing in input
Fetching more results
Fetched 116 results


In [20]:
build_job_search_url("software engineer", "United States", start=len(data))

'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=software%2Bengineer&location=United+States&start=116'

In [15]:
from pydantic import BaseModel, HttpUrl
from typing import Optional

class JobListing(BaseModel):
    title: str
    company: str
    address: str
    timeAdded: str
    jobUrl: HttpUrl
    companyUrl: HttpUrl
    salary: Optional[str]

for result in data:
    job = JobListing(**result)
    print(job.model_dump())

{'title': 'New Grad Software Engineer (2025)', 'company': 'Netflix', 'address': 'Los Gatos, CA', 'timeAdded': '2024-09-12', 'jobUrl': Url('https://www.linkedin.com/jobs/view/new-grad-software-engineer-2025-at-netflix-4023247447'), 'companyUrl': Url('https://www.linkedin.com/company/netflix'), 'salary': None}
{'title': 'Software Engineer I', 'company': 'Nike', 'address': 'Beaverton, OR', 'timeAdded': '2024-09-24', 'jobUrl': Url('https://www.linkedin.com/jobs/view/software-engineer-i-at-nike-4032370167'), 'companyUrl': Url('https://www.linkedin.com/company/nike'), 'salary': None}
{'title': 'Software Engineer (University Grad)', 'company': 'Meta', 'address': 'Menlo Park, CA', 'timeAdded': '2024-09-27', 'jobUrl': Url('https://www.linkedin.com/jobs/view/software-engineer-university-grad-at-meta-4034901235'), 'companyUrl': Url('https://www.linkedin.com/company/meta'), 'salary': None}
{'title': 'Software Engineer, Pay Anyone (New Grad)', 'company': 'Chime', 'address': 'San Francisco, CA', 'ti

In [16]:
len(data)

116

In [17]:
import pandas as pd

In [21]:
jobs_df = pd.DataFrame(data).drop_duplicates(subset=["jobUrl"])
jobs_df

,title,company,address,timeAdded,jobUrl,companyUrl,salary
0,New Grad Software Engineer (2025),Netflix,"Los Gatos, CA",2024-09-12,https://www.linkedin.com/jobs/view/new-grad-so...,https://www.linkedin.com/company/netflix,None
1,Software Engineer I,Nike,"Beaverton, OR",2024-09-24,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/nike,None
2,Software Engineer (University Grad),Meta,"Menlo Park, CA",2024-09-27,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/meta,None
3,"Software Engineer, Pay Anyone (New Grad)",Chime,"San Francisco, CA",2024-09-26,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/chime-card,None
4,"Software Engineer, Intern/Co-op",Meta,"New York, NY",2024-09-27,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/meta,None
5,Software Engineer (New Grad),Genesis Therapeutics,"Burlingame, CA",2024-09-17,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/genesistherap...,None
6,Software Engineer (New Grad),Genesis Therapeutics,United States,2024-09-17,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/genesistherap...,None
7,Software Engineer - Frontend,Julius AI,"San Francisco, CA",2024-09-26,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/julius-ai,None
8,"Software Engineer, Intern/Co-op",Meta,"Seattle, WA",2024-09-27,https://www.linkedin.com/jobs/view/software-en...,https://www.linkedin.com/company/meta,None
9,University Grad Software Engineer (USA),Pinterest,United States,2024-09-18,https://www.linkedin.com/jobs/view/university-...,https://www.linkedin.com/company/pinterest,None


In [23]:
len(data), jobs_df["jobUrl"].nunique()

(116, 58)

In [24]:
jobs_df[["company", "companyUrl"]].value_counts()

company                    companyUrl                                                
Adobe                      https://www.linkedin.com/company/adobe                        10
PayPal                     https://www.linkedin.com/company/paypal                        6
Meta                       https://www.linkedin.com/company/meta                          5
Visa                       https://www.linkedin.com/company/visa                          3
Genesis Therapeutics       https://www.linkedin.com/company/genesistherapeutics           2
Slack                      https://www.linkedin.com/company/tiny-spec-inc                 2
Netflix                    https://www.linkedin.com/company/netflix                       2
WeRide                     https://www.linkedin.com/company/werideai                      2
Cisco                      https://www.linkedin.com/company/cisco                         2
PlanIT Group, LLC          https://www.linkedin.com/company/planit-group-llc          

In [25]:
job_results = await linkedin.scrape_job_search(
    keyword="software engineer",
    location="Seattle",
    max_pages=1
)

pprint(job_results)

2024-09-30 14:28:03.313 | INFO     | scrapfly_scrapers.linkedin:scrape_job_search:192 - scraped the first job page, 0 more pages
2024-09-30 14:28:09.827 | SUCCESS  | scrapfly_scrapers.linkedin:scrape_job_search:203 - scraped 60 jobs from Linkedin job search


[{'address': 'Seattle, WA',
  'company': 'Stripe',
  'companyUrl': 'https://www.linkedin.com/company/stripe',
  'jobUrl': 'https://www.linkedin.com/jobs/view/software-engineer-new-grad-at-stripe-4008209636',
  'salary': None,
  'timeAdded': '2024-09-14',
  'title': 'Software Engineer, New Grad'},
 {'address': 'Bellevue, WA',
  'company': 'Databricks',
  'companyUrl': 'https://www.linkedin.com/company/databricks',
  'jobUrl': 'https://www.linkedin.com/jobs/view/software-engineer-new-grad-2025-start-at-databricks-3992684340',
  'salary': None,
  'timeAdded': '2024-09-17',
  'title': 'Software Engineer - New Grad (2025 Start)'},
 {'address': 'Seattle, WA',
  'company': 'Rover.com',
  'companyUrl': 'https://www.linkedin.com/company/roverdotcom',
  'jobUrl': 'https://www.linkedin.com/jobs/view/software-engineer-search-at-rover-com-3985417406',
  'salary': None,
  'timeAdded': '2024-07-27',
  'title': 'Software Engineer, Search'},
 {'address': 'Bellevue, WA',
  'company': 'The Trade Desk',
 

In [27]:
pd.DataFrame(job_results).company.value_counts()

company
SynergisticIT               12
Adobe                        8
Microsoft                    5
Stripe                       4
Rover.com                    3
Docusign                     2
Meta                         2
Uber                         2
Slack                        2
Amazon                       2
Databricks                   2
The Trade Desk               1
DoorDash                     1
Avispa                       1
Nintendo                     1
Ponto                        1
Visa                         1
Tatango                      1
Blueprint                    1
SoFi                         1
Aerostrat                    1
Qumulo                       1
HiCoder                      1
Roboto AI                    1
Walgreens                    1
CloudKitchens                1
University of Washington     1
Name: count, dtype: int64